### Họ và tên sinh viên:
### Đinh Việt Anh - 19021217
### Trần Thành Long - 19021323

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/visim400/visim-400.txt
/kaggle/input/w2v-150-vn/W2V_150.txt
/kaggle/input/visim400csv/visim-40401.csv


# Tiền xử lý dữ liệu

In [2]:

words = []
vectors = []

with open('../input/w2v-150-vn/W2V_150.txt') as f:
    cnt = 0
    for line in f.readlines():
        if cnt <=1: 
            cnt+=1
            continue
        word, vector = line.split(" ", 1)
        words.append(word)
        vectors.append(vector)

print(f"length of words and vectors: {len(words)}, {len(vectors)}")



length of words and vectors: 77021, 77021


In [3]:
dict_word_2_vec = dict(zip((words), (vectors)))



In [4]:
data = {
    'Word': words,
    'Vector': vectors
}
df = pd.DataFrame(data)

In [5]:
df.head(100)

,Word,Vector
0,những,0.2901038 -0.2898787 0.4169351 1.93689 0.4440...
1,tiết_lộ,-1.308045 0.9506664 2.3402 -0.4334157 -0.2442...
2,thú_vị,2.561394 2.765394 -2.260592 1.895223 1.02471 ...
3,về,-1.414575 -0.5932151 -1.40398 -1.300985 0.946...
4,wallpaper,0.2554106 -1.367197 -0.6043473 0.9219519 0.15...
...,...,...
95,nổi,1.657677 2.757062 0.6732467 -0.5795736 2.2931...
96,thời_gian,-0.4021353 0.1794934 0.3150153 -1.010475 -1.4...
97,bức_ảnh,0.8383942 -0.9070975 -1.715766 -0.8977667 -0....
98,cho_là,-0.6047257 -0.2831929 0.2797906 0.6729242 -1....


# 1. Cosine Similarity

In [6]:
# import csv
# import re
# with open('../input/visim400/visim-400.txt') as f:
#     with open ('visim-4040.txt', 'w') as csv_file:
#         for line in f.readlines():
#             print(line)
#             l = re.sub("\s+", ",", line.strip())
#             print(l)
#             csv_file.write(l+"\n")

            

In [7]:
df_visim400 = pd.read_csv('../input/visim400csv/visim-40401.csv')



In [8]:
df_visim400

,Word1,Word2,POS,Sim1,Sim2,STD
0,biến,ngập,V,3.13,5.22,0.72
1,nhà_thi_đấu,nhà,N,3.07,5.12,1.18
2,động,tĩnh,V,0.60,1.00,0.95
3,khuyết,ưu,N,0.20,0.33,0.40
4,cõi_tục,cõi_âm,N,0.60,1.00,0.95
...,...,...,...,...,...,...
395,lình_xình,nặng_tình,A,1.33,2.22,1.14
396,người_làm,người_bị_hại,N,2.20,3.67,0.83
397,cõi_tục,trần_gian,N,5.40,9.00,0.71
398,chần_chừ,lảo_đảo,V,3.20,5.33,0.98


In [9]:
from numpy import dot
from numpy.linalg import norm
from scipy import spatial
# cos_sim = dot(a, b)/(norm(a)*norm(b))

def cosine_similarity(word1, word2):
    return dot(word1, word2)/(norm(word1)*norm(word2))

def cosine_similarity_2(word1, word2):
    return 1 - spatial.distance.cosine(word1, word2)

    

In [10]:
res_cosine_similarity = []

def get_vec_from_word(word):
    if dict_word_2_vec.get(word) is None:
        return None
    res = (dict_word_2_vec.get(word).split())
    return [float(l) for l in res ]

for idx in df_visim400.index:
    
    word1 = df_visim400['Word1'][idx]
    word2 = df_visim400['Word2'][idx]
    print(word1,word2)

    vec1 = get_vec_from_word(word1)
    vec2 = get_vec_from_word(word2)
    if vec1 is None or vec2 is None:
        res_cosine_similarity.append("Not have one of 2 words")
        print(f"Cosine similarity of {word1} and {word2} is: Not have one of 2 words")
        continue
    cosine_si = cosine_similarity(vec1, vec2)
    cosine_si2 = cosine_similarity_2(vec1, vec2)
    res_cosine_similarity.append(str(cosine_si))
    print(f"Cosine similarity of {word1} and {word2} is: {cosine_si}")
    print(f"Cosine similarity of {word1} and {word2} is: {cosine_si2}")

biến ngập
Cosine similarity of biến and ngập is: -0.0049123394696700354
Cosine similarity of biến and ngập is: -0.004912339469669957
nhà_thi_đấu nhà
Cosine similarity of nhà_thi_đấu and nhà is: 0.08252318329211769
Cosine similarity of nhà_thi_đấu and nhà is: 0.08252318329211772
động tĩnh
Cosine similarity of động and tĩnh is: 0.2770859598682775
Cosine similarity of động and tĩnh is: 0.27708595986827755
khuyết ưu
Cosine similarity of khuyết and ưu is: 0.17679862835626717
Cosine similarity of khuyết and ưu is: 0.17679862835626714
cõi_tục cõi_âm
Cosine similarity of cõi_tục and cõi_âm is: Not have one of 2 words
thủ_pháp biện_pháp
Cosine similarity of thủ_pháp and biện_pháp is: 0.402366129194306
Cosine similarity of thủ_pháp and biện_pháp is: 0.40236612919430603
kết_duyên thành_hôn
Cosine similarity of kết_duyên and thành_hôn is: 0.46300840201407223
Cosine similarity of kết_duyên and thành_hôn is: 0.46300840201407223
cấp_tiến bảo_thủ
Cosine similarity of cấp_tiến and bảo_thủ is: 0.2569470

In [11]:
df_visim400["cosine_similarity"] = res_cosine_similarity

In [12]:
df_visim400.head(10)

,Word1,Word2,POS,Sim1,Sim2,STD,cosine_similarity
0,biến,ngập,V,3.13,5.22,0.72,-0.0049123394696700354
1,nhà_thi_đấu,nhà,N,3.07,5.12,1.18,0.08252318329211769
2,động,tĩnh,V,0.60,1.00,0.95,0.2770859598682775
3,khuyết,ưu,N,0.20,0.33,0.40,0.17679862835626717
4,cõi_tục,cõi_âm,N,0.60,1.00,0.95,Not have one of 2 words
5,thủ_pháp,biện_pháp,N,4.13,6.88,1.26,0.402366129194306
6,kết_duyên,thành_hôn,V,5.27,8.78,1.06,0.46300840201407223
7,cấp_tiến,bảo_thủ,A,0.87,1.45,1.15,0.25694700889961236
8,nước_lớn,nguy_hiểm,N,1.07,1.78,1.12,0.18519202240211616
9,hoa,nhị,N,2.27,3.78,0.93,0.22306960452548538


# 2. K-nearest words

In [13]:
def k_nearest_words(word:str, k:int) -> list:
    vec = get_vec_from_word(word)
#     print(vec)
    title = "Cosine_similarity"
    df[title] = df["Word"].apply(lambda word: cosine_similarity(get_vec_from_word(word), vec))
    print(df.sort_values(by=['Cosine_similarity'],ascending=False)["Word"].tolist()[0:k])
    
        

In [14]:
k_nearest_words("thông_minh", 5)
# df.sort_values(by=['Cosine_similarity'],ascending=False)

['thông_minh', 'nhạy_bén', 'khôn_ngoan', 'nhanh_nhạy', 'sáng_suốt']


In [15]:
print(df.sort_values(by=['Cosine_similarity'],ascending=False)["Word"].tolist()[0:5])

['thông_minh', 'nhạy_bén', 'khôn_ngoan', 'nhanh_nhạy', 'sáng_suốt']
